# Environment Setup
This notebook uses a Poetry environment.
1.  Ensure you have the `erm-geemap` kernel selected.
    *   In VS Code, click "Select Kernel" at the top right and choose **Python (erm-geemap)**.
2.  If you haven't set up the environment yet, run:
    ```bash
    poetry install
    poetry run python -m ipykernel install --user --name=erm-geemap --display-name "Python (erm-geemap)"
    ```

## 1. Install and Import Libraries
Import the necessary libraries for the analysis.

In [1]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import datetime

## 2. Initialize Earth Engine
Authenticate and initialize the Earth Engine API.

In [2]:
# Initialize Earth Engine
try:
    ee.Initialize(project='noble-truck-479712-a1')
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='noble-truck-479712-a1')


Successfully saved authorization token.


## 3. Visualization Parameters
Define color palettes and visualization parameters for the map layers.

In [3]:
# Visualization Parameters
visRain = {'min': 1, 'max': 100, 'palette': ['cccccc', 'f9f3d5', 'dce2a8', 'a8c58d', '77a87d', 'ace8f8', '4cafd9', '1d5ede', '001bc0', '9131f1', 'e983f3', 'f6c7ec']}
visFlood = {'min': 0, 'max': 9, 'palette': ['97D700', 'FFEDA0', 'FFEDA0', 'FFEDA0', 'FEB24C', 'FEB24C', 'FEB24C', 'F03B20', 'F03B20', 'F03B20']}
visExtreme = {'min': 1, 'max': 4, 'palette': ['ffffcc', 'a1dab4', '41b6c4', '225ea8']}
visLikelihood = {'min': 1, 'max': 3, 'palette': ['f7fcb9', 'addd8e', '31a354']}
visPopulation = {'min': 0, 'max': 50.0, 'palette': ['yellow', 'orange', 'red']}

# Region configuration
REGIONS = {
    'Indonesia': {
        'name': 'Indonesia',
        'center': [-2.5, 118],
        'zoom': 5,
        'adm0_filter': ee.Filter.eq('ADM0_NAME', 'Indonesia')
    },
    'Nepal': {
        'name': 'Nepal',
        'center': [28.3, 84.1],
        'zoom': 7,
        'adm0_filter': ee.Filter.eq('ADM0_NAME', 'Nepal')
    }
}

# ========== SELECT YOUR REGION HERE ==========
SELECTED_REGION = 'Indonesia'  # Change to 'Nepal' or 'Indonesia'
# =============================================

# Get region config
region_config = REGIONS[SELECTED_REGION]
print(f"Selected region: {region_config['name']}")

# Static layers based on selected region
adm0 = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(region_config['adm0_filter'])
mask_land = ee.Image.constant(1).clip(adm0)
outline_prov = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(region_config['adm0_filter'])

Selected region: Indonesia


## 4. Helper Functions
Define functions to retrieve threshold data and calculate extreme rainfall.

In [ ]:
# Helper functions for data retrieval and processing

def getThresholdImage(p, numday=5):
    q = {
        'P50': 'q0500_2yr',
        'P80': 'q0800_5yr',
        'P90': 'q0900_10yr',
        'P96': 'q0960_25yr'
    }[p]
    path = f'users/bennyistanto/datasets/raster/extremerainfall/threshold/idn_cli_{numday}day_precipthreshold_{q}_imerg_wfp'
    return ee.Image(path)

def getSlopeImage(month, numday=5):
    m = ['', '01_jan', '02_feb', '03_mar', '04_apr', '05_may', '06_jun',
         '07_jul', '08_aug', '09_sep', '10_oct', '11_nov', '12_dec'][month]
    path = f'users/bennyistanto/datasets/raster/extremerainfall/slope/idn_cli_day{numday}_{m}_slope_imerg'
    return ee.Image(path)

def getInterceptImage(month, numday=5):
    m = ['', '01_jan', '02_feb', '03_mar', '04_apr', '05_may', '06_jun',
         '07_jul', '08_aug', '09_sep', '10_oct', '11_nov', '12_dec'][month]
    path = f'users/bennyistanto/datasets/raster/extremerainfall/intercept/idn_cli_day{numday}_{m}_intercept_imerg'
    return ee.Image(path)

def getImergImage(dt, numday=5):
    hours = -24 * numday
    return ee.ImageCollection("NASA/GPM_L3/IMERG_V06") \
        .select('precipitationCal') \
        .filterDate(dt.advance(hours, 'hour'), dt) \
        .sum().float()

def getGfsImage(dt, numday=5):
    # Grab forecast data for hours 6,12,18,24 (Day1) ...
    hours = []
    for i in range(numday):
        for j in range(4):
            h = (i * 24) + ((j + 1) * 6)
            hours.append(h)
            
    return ee.ImageCollection('NOAA/GFS0P25') \
        .select('total_precipitation_surface') \
        .filterDate(dt, dt.advance(6, 'hour')) \
        .filter(ee.Filter.inList('forecast_hours', hours)) \
        .sum() \
        .reproject(crs='EPSG:4326', scale=11132) # Approximate scale matching IMERG

def getRainfallExtremeImage(date_str, numday=5, imgBase=None):
    dt = ee.Date(date_str)
    month = dt.get('month').getInfo()
    
    # If imgBase is not provided, default to IMERG (NRT)
    if imgBase is None:
        imgBase = getImergImage(dt, numday)
    
    # Get Thresholds
    P50 = getThresholdImage('P50', numday)
    P80 = getThresholdImage('P80', numday)
    P90 = getThresholdImage('P90', numday)
    P96 = getThresholdImage('P96', numday)
    
    # Slope & Intercept
    imgSlope = getSlopeImage(month, numday)
    imgIntercept = getInterceptImage(month, numday)
    
    # Impact
    imgImpact = imgBase.expression(
        'R>P96?4:(R>P90?3:(R>P80?2:(R>P50?1:0)))', {
            'R': imgBase,
            'P50': P50,
            'P80': P80,
            'P90': P90,
            'P96': P96
        }
    )
    
    # Probability
    imgProbBase = imgBase.expression(
        '1/(1+(e **(-((S*R)+I))))', {
            'e': 2.71828,
            'R': imgBase,
            'S': imgSlope,
            'I': imgIntercept
        }
    )
    
    imgProb = imgProbBase.expression(
        'V<0.6?0:(V<0.8?1:(V<=1?2:0))', {
            'V': imgProbBase
        }
    )
    
    # Flood Alert (ERTF)
    # ertf = (x*3) + y + 1
    # x = impact, y = prob
    imgReBase = imgImpact.multiply(3).add(imgProb).add(1)
    
    # Remap to final classes
    # 1-15 -> -1 to 9
    from_vals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    to_vals = [-1, -1, -1, 0, 0, 1, 0, 2, 4, 3, 5, 7, 6, 8, 9]
    imgRe = imgReBase.remap(from_vals, to_vals)

    # Remap to 3 classes (Low, Medium, High)
    from_vals_cls = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
    to_vals_cls = [-1, -1, -1, 0, 0, 1, 0, 1, 2, 1, 2, 3, 2, 3, 3]
    imgReClass = imgReBase.remap(from_vals_cls, to_vals_cls)
    
    return {
        'rainfall': imgBase,
        'alert': imgRe.updateMask(imgRe.gte(0)),
        'impact': imgImpact.updateMask(imgImpact.gt(0)),
        'prob': imgProb.updateMask(imgProb.gt(0)),
        'reClass': imgReClass.updateMask(imgReClass.gte(0))
    }

## 5. Impact Analysis Functions
Define functions to calculate affected population and cropland.

In [ ]:
# Impact Analysis Functions
def get_affected_population(re_class_image, geometry):
    ghsl = ee.Image("JRC/GHSL/P2016/POP_GPW_GLOBE_V1/2015").clip(geometry)
    stats = {}
    
    # Iterate through alert classes: 1 (Yellow), 2 (Orange), 3 (Red)
    for i in range(1, 4):
        # Mask population where remapped class equals i
        affected = ghsl.updateMask(re_class_image.eq(i))
        
        # Calculate sum of affected population
        result = affected.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geometry,
            scale=1000, # GHSL resolution is 250m, but using 1000m for speed in demo
            maxPixels=1e9
        )
        stats[f'Alert {i}'] = result.get('population_count').getInfo()
        
    return stats

def get_affected_cropland(re_class_image, geometry):
    # MODIS Land Cover
    mcd12q1 = ee.Image('MODIS/006/MCD12Q1/2019_01_01').select("LC_Type1").clip(geometry)
    # Crop mask: 12 (Croplands), 14 (Cropland/Natural Vegetation Mosaic)
    cropmask = mcd12q1.eq(12).Or(mcd12q1.eq(14))
    cropland = mcd12q1.updateMask(cropmask)
    
    stats = {}
    
    # Iterate through alert classes: 1 (Yellow), 2 (Orange), 3 (Red)
    for i in range(1, 4):
        # Mask cropland where remapped class equals i
        affected = cropland.updateMask(re_class_image.eq(i))
        
        # Calculate area in Hectares
        area_image = affected.gte(0).multiply(ee.Image.pixelArea()).divide(10000)
        
        result = area_image.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=geometry,
            scale=500,
            maxPixels=1e9
        )
        stats[f'Alert {i}'] = result.get('LC_Type1').getInfo()
        
    return stats

## 6. Create Interactive Map
Initialize the map and add static layers.

In [6]:
# Create the interactive map centered on selected region
Map = geemap.Map(center=region_config['center'], zoom=region_config['zoom'])
Map.add_basemap('HYBRID')

# Add static layers
Map.addLayer(mask_land, {}, 'Land Mask', False)
Map.addLayer(outline_prov, {}, 'Provincial Boundary', False)

# Add widgets
style = {'description_width': 'initial'}
date_slider = widgets.DatePicker(description='Select Date:', disabled=False, style=style)
display(date_slider)

Map

DatePicker(value=None, description='Select Date:', step=1, style=DescriptionStyle(description_width='initial')…

Map(center=[-2.5, 118], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', t…

## 7. Run Analysis
Run the extreme rainfall detection and impact analysis for a specific date.

In [7]:
# Run the analysis
# For demonstration, we use a fixed date or the current date
# In a real app, this would be triggered by the slider
analysis_date = '2021-04-04' # Example date (Cyclone Seroja)
dt = ee.Date(analysis_date)

print(f"Running analysis for {analysis_date}...")

# 1. Near Real-Time (IMERG)
print("1. Running Near Real-Time (NRT) Analysis...")
nrt_rain = getImergImage(dt)
nrt_result = getRainfallExtremeImage(analysis_date, imgBase=nrt_rain)

# 2. Forecast (GFS)
print("2. Running Forecast (FCT) Analysis...")
fct_rain = getGfsImage(dt)
fct_result = getRainfallExtremeImage(analysis_date, imgBase=fct_rain)

# Add analysis layers to map (static layers already added in previous cell)
Map.addLayer(nrt_result['rainfall'], visRain, 'NRT Rainfall', False)
Map.addLayer(nrt_result['alert'], visFlood, 'NRT Flood Alert', True)
Map.addLayer(fct_result['rainfall'], visRain, 'FCT Rainfall', False)
Map.addLayer(fct_result['alert'], visFlood, 'FCT Flood Alert', False)

# Calculate impact (NRT)
print("\n--- NRT Impact Analysis ---")
nrt_pop = get_affected_population(nrt_result['reClass'], outline_prov.geometry())
nrt_crop = get_affected_cropland(nrt_result['reClass'], outline_prov.geometry())

print("Affected Population:")
for k, v in nrt_pop.items():
    print(f"  {k}: {v:,.0f}")

print("Affected Cropland (Hectares):")
for k, v in nrt_crop.items():
    print(f"  {k}: {v:,.2f}")

# Calculate impact (Forecast)
print("\n--- Forecast Impact Analysis ---")
fct_pop = get_affected_population(fct_result['reClass'], outline_prov.geometry())
fct_crop = get_affected_cropland(fct_result['reClass'], outline_prov.geometry())

print("Affected Population:")
for k, v in fct_pop.items():
    print(f"  {k}: {v:,.0f}")

print("Affected Cropland (Hectares):")
for k, v in fct_crop.items():
    print(f"  {k}: {v:,.2f}")

Running analysis for 2021-04-04...
1. Running Near Real-Time (NRT) Analysis...
2. Running Forecast (FCT) Analysis...

--- NRT Impact Analysis ---
Affected Population:
  Alert 1: 683,182
  Alert 2: 887,949
  Alert 3: 219,964
Affected Cropland (Hectares):
  Alert 1: 357,863.39
  Alert 2: 775,865.68
  Alert 3: 164,297.84

--- Forecast Impact Analysis ---
Affected Population:
  Alert 1: 1,502
  Alert 2: 12,260
  Alert 3: 3,185
Affected Cropland (Hectares):
  Alert 1: 2,902.97
  Alert 2: 41,126.37
  Alert 3: 12,338.17


## 8. Export Map to HTML
Save the interactive map as an HTML file.

In [8]:
# Export the map to an HTML file
Map.to_html(filename="erm_map.html", title="Extreme Rainfall Monitoring", layer_control=True)

Exception: embed_snippet() got an unexpected keyword argument 'layer_control'